In [1]:
import unittest
import maboss
import math

In [2]:
class MaBoSSTestCase(unittest.TestCase):
    
    def __init__(self, sim, verbose = True):
        unittest.TestCase.__init__(self)
        self.Simulation = sim.copy()
        self.Old_sim = (self.Simulation).copy()
        self.Old_result = None 
        self.New_sim = (self.Simulation).copy()     
        self.New_result = None
        self.VERBOSE = verbose
    
    #call at the end of the tests
    def Reset_simulations(self):
        self.Old_sim = (self.Simulation).copy()
        self.Old_result = None
        self.New_sim = (self.Simulation).copy()
        self.New_result = None
    
    
    def Check_nodes(self, state):
        for node in state:
            if node not in self.Simulation.network.keys():
                print(node, 'is not present in the network.')
                return False
        
        return True
    
    
    #set output for save time when you run a simulation   
    def Set_output(self, output):
        self.Old_sim.network.set_output(output)
        self.New_sim.network.set_output(output)
              
        
    def Set_initial_conditions(self, initial_conditions):
        
        if not initial_conditions: print('Care, you did not insert initial conditions!')
        
        if type(initial_conditions) is dict:
            for node in initial_conditions:
                self.Old_sim.network.set_istate(node, initial_conditions[node])
                self.New_sim.network.set_istate(node, initial_conditions[node])
        #input: list: list[0]=list of node, list[1] dictionary of prob        
        elif type(initial_conditions) is list:
            self.Old_sim.network.set_istate(initial_conditions[0], initial_conditions[1])
            self.New_sim.network.set_istate(initial_conditions[0], initial_conditions[1])
        
        else: print('Invalid initial condition!')
            
        return
    
    
    #mutate New_simulation if a list of mutations is given
    def Mutate_simulation(self, mutations={}):
        if mutations: 
            for n in mutations:
                (self.New_sim).mutate(n,mutations[n])
                
        else: print('Care, you did not insert mutations!')
            
        
    def Run_both_simulations(self):
        self.Old_result = self.Old_sim.run()
        self.New_result = self.New_sim.run()
        
    #return the states of a sim with their probability
    def Get_probtraj_states(self, sim_name):
        if sim_name == 'New': result = (self.New_result).get_last_states_probtraj()
        elif sim_name == 'Old': result = (self.Old_result).get_last_states_probtraj()
        
        states = result.columns    
        probability_states = {}  # dict----> key:probability  value:state in list form
    
        for state in states:
            prob = result[state].values[0]
            probability_states[prob] = state.split(' -- ')
            
        return probability_states
    
    #return the stable states of a sim with their probability
    def Get_stable_states(self, sim_name):
        if sim_name == 'New': s_states_table = self.New_result.get_fptable()
        elif sim_name == 'Old': s_states_table = self.Old_result.get_fptable()
            
        s_states = s_states_table['State']
        s_states = [state.split(' -- ') for state in s_states]
        s_states_prob = s_states_table['Proba']
        
        probability_states = dict(zip(s_states_prob,s_states))
        
        return probability_states
            
    #return a dictionari with the states satisfing the condition with probability as key     
    def Check_for_a_state(self, kind, condition={}, all_states={}):
        
        #check if I'm looking for <nil> state only for last prob traj :CHECK CASE IN WICH ALL STATES ARE 0
        if ( kind == 'last' and all(i == 0 for i in condition.values()) ): 
            active_nodes = ['<nil>']
            inactive_nodes = []
        
        else:
            active_nodes = [x for x in condition if condition[x]==1]
            inactive_nodes = [x for x in condition if condition[x]==0]
        
        resulting_states = all_states.copy()    #states that s
        
        for node in active_nodes:
            resulting_states = {x: resulting_states[x] for x in resulting_states if node in resulting_states[x]}
            
        for node in inactive_nodes:
            resulting_states = {x: resulting_states[x] for x in resulting_states if node not in resulting_states[x]}
        
        if resulting_states: return resulting_states  #if there is not match the probability is 0
        else: return None #{0: condition}
        
        
            
    def Truncate(self, number, digits) -> float:
        stepper = 10.0 ** digits
        return math.trunc(stepper * number) / stepper
    
    #print the dictionary: prob_ state in a nice form
    def Print_states(self, states):
        if states == None: print('No one')
        else:
            for prob in states:
                print('\nProbability = ', prob, '\nState: ', str(states[prob]))
        
        return
    
    
    def Compare(self, Old_p, New_p, direction, message, digits):
        Old_p = self.Truncate(Old_p, digits)
        New_p = self.Truncate(New_p, digits)
              
        #INCREASE?
        if direction == self.INCREASE: 
            try:
                self.assertTrue(New_p > Old_p)
                if self.VERBOSE: print("True! ", message)
                else: print('... OK')
            
            except Exception as e:
                if self.VERBOSE: print("False! ", message)
                else: print(e)
                
        #DECREASE?   
        elif direction == self.DECREASE:
            try:
                self.assertTrue(New_p < Old_p)
                if self.VERBOSE: print("True! ", message)
                else: print('... OK')
            
            except Exception as e:
                if self.VERBOSE: print("False! ", message)
                else: print(e)
                
        #STAY THE SAME?
        elif direction == self.CHANGE: 
            try:
                self.assertAlmostEqual(Old_p, New_p)
                if self.VERBOSE: print("True! ", message)
                else: print('... OK')
            
            except Exception as e:
                if self.VERBOSE: print("False! ", message)
                else: print(e)
            
           
        return   
    
    
    ##################################################################################################################
    ############################################ ASSERTION FUNCTIONS #################################################
    ##################################################################################################################
    
    
    
    def AssertStateProbability(self, mutations, I_C, state, direction, digits = 4):
        
        if not self.Check_nodes(state): return
        
        output = list(state.keys())
        
        self.Set_output(output)             #set the output for Old/New_sim 
        self.Set_initial_conditions(I_C)    #set initial conditions
        self.Mutate_simulation(mutations)   #mutate New_sim
        
        self.Run_both_simulations()
        
        all_Old_states = self.Get_probtraj_states('Old')  #dict: key:stateProb  value:state (list form)
        all_New_states = self.Get_probtraj_states('New')
        
        Old_states = self.Check_for_a_state('last', state, all_Old_states)  #there should be prob_state:state_list_form 
        New_states = self.Check_for_a_state('last', state, all_New_states)  #if prob is not 0  
        
        
        if Old_states == None: Old_state_prob = 0
        else: Old_state_prob = list(Old_states.keys())[0]                 #if possible invert keys and value
            
        if New_states == None: New_state_prob = 0
        else: New_state_prob = list(New_states.keys())[0]
        
        #check it is all fine
        if (Old_states != None and New_states != None and len(Old_states)!=1 and len(Old_states)!=1):
            print('ERROR, TO MUTCH STATES')
            return
         
        message = '\nThe new probability of reaching the state is: {}' \
        ' \nThe old one is: {} \nThe comparison was made with {} digits'.format(New_state_prob, Old_state_prob, digits)
        
        self.Compare(Old_state_prob, New_state_prob, direction, message, digits) #ad an output!!!!!
        
        self.Reset_simulations()
    
        return
    
    
    def AssertStableStateProbability(self, mutations, I_C, state, direction, digits = 4):
         
        if not self.Check_nodes(state): return
        
        self.Set_initial_conditions(I_C)    #set initial conditions
        self.Mutate_simulation(mutations)   #mutate New_sim
        
        self.Run_both_simulations()
        
        all_Old_stable_states = self.Get_stable_states('Old')
        all_New_stable_states = self.Get_stable_states('New')
        
        Old_states = self.Check_for_a_state('stable', state, all_Old_stable_states)
        New_states = self.Check_for_a_state('stable', state, all_New_stable_states)     
        
        Old_state_prob = 0
        if Old_states != None: 
            for prob in Old_states.keys():
                Old_state_prob += prob
        
        New_state_prob = 0
        if New_states != None: 
            for prob in New_states.keys():
                New_state_prob += prob
            
        message = '\nThe new probability of reaching the stable state is: {}' \
        ' \nThe old one is: {} \nThe comparison was made with {} digits'.format(New_state_prob, Old_state_prob, digits)
        
        self.Compare(Old_state_prob, New_state_prob, direction, message, digits)
            
        self.Reset_simulations()
        
        return
    
    
    #NAME?????
    #maBoss only find ss with prob >0?
    #assert if given some condition (nodes dictionaries) other nodes has always the same values
    def AssertStableStates(self, mutations, condition, nodes_expected):
        
        if not self.Check_nodes(condition): return
        if not self.Check_nodes(nodes_expected): return
        #for stable states don't need outputs
        self.Mutate_simulation(mutations)   #mutate New_sim
        
        self.New_result = self.New_sim.run()
        
        all_stable_states = self.Get_stable_states('New')
        #states satisfying the input condition
        states_satisfying_condition = self.Check_for_a_state('stable', condition, all_stable_states)
        #selected states satisfying both condition and nodes_expected
        if states_satisfying_condition != None:
            selected_states = self.Check_for_a_state('stable', nodes_expected, states_satisfying_condition)
        
        else:
            print("Not even one stable state satisfy: ", condition)
            return
        
        try:
            self.assertEqual(states_satisfying_condition, selected_states)
            if self.VERBOSE: print("True! \nAll the states that satisfy: ", condition, " have: ", nodes_expected)
            else: print("... OK")
                
        except Exception as e: 
            if self.VERBOSE: 
                print('False! \nThe states with ', condition, ' are : ') 
                self.Print_states(states_satisfying_condition)
                print('\nOf these, those with ', nodes_expected, ' are: ')                        
                self.Print_states(selected_states)                  
                                         
            else: print(e)
            
        
        self.Reset_simulations()
        
        return
    
    
    #IT IS CLEAR??
    #compare the probability of reaching a state with prob (1 ask INCREASE, -1 DECREASE, 0 IS EQUAL)
    def CompareStableStateProbability(self, mutations, state, direction, reference_prob = 0, digits = 4):
         
        if not self.Check_nodes(state): return    
        
        self.Mutate_simulation(mutations)          #mutate New_sim
        
        self.New_result = self.New_sim.run()
        
        all_New_stable_states = self.Get_stable_states('New')
        
        New_states = self.Check_for_a_state('stable', state, all_New_stable_states)     
        
        New_state_prob = 0
        if New_states != None: 
            for prob in New_states.keys():
                New_state_prob += prob
            
        message = '\nThe probability of reaching the state is: {}'.format(New_state_prob)
        print('The reference probability is: ', reference_prob)
        self.Compare(reference_prob, New_state_prob, direction, message, digits)
            
        self.Reset_simulations()
        
        return
        
        
        
    INCREASE = 'increase'
    DECREASE = 'decrease'
    CHANGE = 'stable'                      #actually it is: does not change  
    

# Lympho test

In [3]:
fast_sim = maboss.load("Four_cycle.bnd", "Four_cycle_FEscape.cfg")

In [4]:
test_fast_sim = MaBoSSTestCase(fast_sim)

# Functions:

## AssertStateProbability(mutations, I_C, state, direction, digits = 4):
Assert if a mutation alter the state probability to be active, pass as arguments:

* mutations: dictionary of mutations
* I_C: initial condiotin: dictionary {node: [p, 1-p]} or list: [ [node1, node2], {proba_dict} ] as in Maboss
* state (the state you want to observe): dictionary of nodes; 1 if active 0 if inactive
* direction: increase/decrease/stable
* number of digits keep in the approximation

## AssertStableStateProbability(mutations, I_C, state, direction, digits = 4):
Assert if a mutation alter the state probability to be active, pass as arguments:
* mutations: dictionary of mutations
* I_C: initial condiotin: dictionary {node: [p, 1-p]} or list: [ [node1, node2], {proba_dict} ] as in Maboss
* state (the state you want to observe): dictionary of nodes; 1 if active 0 if inactive
* direction: increase/decrease/stable
* number of digits keep in the approximation

## AssertStableStates(mutations, condition, nodes_expected):
Check if all states satisfying a condition present the expected state for some given nodes:
* mutations: dictionary of mutations
* condition of the states you are looking for: dictionary of nodes; 1 if active 0 if inactive
* state of nodes in the states satifyng the condition: dictionary of nodes; 1 if active 0 if inactive


## CompareStableStateProbability(mutations, state, direction, reference_prob = 0, digits = 4):
compare the probability to reach a state with a given probability (0 default):
* mutations: dictionary of mutations
* state (the state you want to observe): dictionary of nodes; 1 if active 0 if inactive
* direction: increase/decrease/stable
* reference_prob: the probability to compare 
* number of digits keep in the approximation

# Examples: 
## AssertStateProbability()

In [5]:
test_fast_sim.AssertStateProbability({'A':'OFF'}, {}, {'A':1}, 'decrease')

Care, you did not insert initial conditions!
True!  
The new probability of reaching the state is: 0 
The old one is: 0.0022 
The comparison was made with 4 digits


In [6]:
test_fast_sim.AssertStateProbability({'A':'OFF'}, {'A': [1,0]}, {'A':1}, 'increase')

False!  
The new probability of reaching the state is: 0 
The old one is: 0.0022 
The comparison was made with 4 digits


In [7]:
test_fast_sim.AssertStateProbability({'B':'ON','A':'OFF'}, {}, {'C':1}, 'increase' )

Care, you did not insert initial conditions!
True!  
The new probability of reaching the state is: 1.0 
The old one is: 0.004 
The comparison was made with 4 digits


In [8]:
test_fast_sim.AssertStateProbability({}, {}, {'A':1, 'B':1, 'C':1}, 'stable')

Care, you did not insert initial conditions!
Care, you did not insert mutations!
True!  
The new probability of reaching the state is: 0.001 
The old one is: 0.001 
The comparison was made with 4 digits


## AssertStableStateProbability()

In [18]:
test_fast_sim.AssertStableStateProbability({'A':'OFF'}, {}, {'A':1}, 'decrease')

Azefz is not present in the network.


In [10]:
test_fast_sim.AssertStableStateProbability({'A':'OFF'}, {'A': [1,0]}, {'A':1}, 'increase')

False!  
The new probability of reaching the stable state is: 0 
The old one is: 0 
The comparison was made with 4 digits


In [11]:
test_fast_sim.AssertStableStateProbability({'B':'ON','A':'OFF'}, {}, {'C':1}, 'increase' )

Care, you did not insert initial conditions!
True!  
The new probability of reaching the stable state is: 1 
The old one is: 0 
The comparison was made with 4 digits


In [12]:
test_fast_sim.AssertStableStateProbability({}, {}, {'A':1, 'B':1, 'C':1}, 'stable')

Care, you did not insert initial conditions!
Care, you did not insert mutations!
True!  
The new probability of reaching the stable state is: 0 
The old one is: 0 
The comparison was made with 4 digits


## AssertStableStates(mutations, condition, nodes_expected):

In [13]:
test_fast_sim.AssertStableStates({}, {'A':0}, {'B':0} )

Care, you did not insert mutations!
True! 
All the states that satisfy:  {'A': 0}  have:  {'B': 0}


In [14]:
test_fast_sim.AssertStableStates({}, {'A':1}, {'B':0} )

Care, you did not insert mutations!
Not even one stable state satisfy:  {'A': 1}


In [15]:
test_fast_sim.AssertStableStates({}, {'A':0}, {'B':1} )

Care, you did not insert mutations!
False! 
The states with  {'A': 0}  are : 

Probability =  0.9962 
State:  ['<nil>']

Of these, those with  {'B': 1}  are: 
No one


In [16]:
test_fast_sim.AssertStableStates({'B':'ON','A':'OFF'}, {'A':0}, {'B':1, 'C':0})

False! 
The states with  {'A': 0}  are : 

Probability =  1 
State:  ['B', 'C']

Of these, those with  {'B': 1, 'C': 0}  are: 
No one


## CompareStableStateProbability(mutations, state, direction, reference_prob = 0, digits = 4)

In [17]:
test_fast_sim.CompareStableStateProbability({'B':'ON','A':'OFF'}, {'B':0}, 'stable', reference_prob = 0.5)

The reference probability is:  0.5
False!  
The probability of reaching the state is: 0
